### Тестовое задание в Лабораторию Касперского Backend

# Python

Задания

 

Задания нужно сделать на Python

#### 1.       Чем плохи деструкторы в питоне? 

In [1]:
class ClassWithId:
    def __init__(self, id):
        self.id = id
        print(f"Ctr Empty class № {self.id}")
        
    def __del__(self):
        print(f"Dtr Empty class № {self.id}")
    

In [10]:
instance_1 = ClassWithId(1)
ref_on_instance_1 = instance_1 # Не копируем а берём ссылку

Ctr Empty class № 1


In [11]:
del instance_1 # по логике должен вызваться деструктор

In [12]:
del ref_on_instance_1 # но на самом деле он вызоветься только когда все ссылки будут удалены

Dtr Empty class № 1


#### 2.       Почему может быть плох import в начале файла? 

1)<br>
К примеру мы напищем<br>
import packetWithSpecificPath.someModule<br>

Модуль лежащий в каком-то специфичном месте, для него нужно предварительно дополнить sys.path<br>

2)<br>
К примеру в нашей рабочей папке (в текущей момент это та в которой лежит этот ноутбук)<br>
(Также это распространяется на какие-то другие модули, не обязательно в текущей рабочей директории)<br>
Может лежать модуль, которому необходима переменная определённая у нас здесь<br>

Положим сейчас в ./ есть модуль mmmodule.py<br>
В котором будет функция<br>
<br>
<code>def fffunction():
    from generalModule import glob
    print(f"i'm need glob == 42, now glob = {glob}")
    if glob == 42:
        print("Cool i got glob = 42")</code>
<br>    
Для этого понадобиться общий класс <code>generalModule.py</code><br>
Чтобы в нём определить переменную <code>var_from_notebook</code><br>

generalModule.py<br>
|-glob = 0<br>
|<br>
|->test_tasks.ipynb<br>
|&nbsp;&nbsp;&nbsp;&nbsp;|-generalModule.glob = 42<br>
|<br>
|->mmmodule.py<br>
|&nbsp;&nbsp;&nbsp;&nbsp;|-fffunction (корректно работает с 42)<br>

In [1]:
import generalModule
generalModule.glob = 42

In [2]:
from mmmodule import fffunction
fffunction()

i'm need glob == 42, now glob = 42
Cool i got glob = 42


#### 3.       Как в питоне «вывалиться» из любой вложенности или рекурсии по аналогу «goto:»? 

In [16]:
class Goto(Exception):
    pass

In [48]:
counter = 0

In [49]:
def recursive_function():
    global counter
    if counter == 10:
        raise Goto
    counter += 1
    recursive_function()

In [50]:
try:
    for i in range(100):
        counter += 1
        for j in range(1000):
            counter += 1
            recursive_function()
    
except(Goto):
    print(f"Handmade goto : counter = {counter}")

Handmade goto : counter = 10


#### 4.       Сервер отдает сообщение один раз (1кб, бинарь). Его необходимо сохранить в базе, нельзя потерять(!). Как бы вы решали такую задачу? 

In [277]:
#!pip install requests

In [278]:
import requests

In [279]:
# Получим не бинарь, а обратимся например к гуглу
res = requests.get("https://www.google.com")
data = res.text

In [280]:
sys.getsizeof(data) # в 16 раз больше 1кб из условия, но я думаю это не страшно

16762

In [281]:
bin_data = ''.join(map(bin, bytearray(data, 'utf-8')))

In [282]:
while sys.getsizeof(bin_data) > 2048:
    bin_data = bin_data[:len(bin_data)//2]

In [283]:
sys.getsizeof(bin_data) # теперь действительно один килобайт

1162

In [284]:
data = bin_data

In [285]:
type(data)

str

In [286]:
# Давайте сохраним данные в файл, чтобы гарантированно не потерять в дальнейшем
with open('quasy_bin_file', 'w') as fh:
    fh.write(data)
    fh.close()

In [287]:
import sqlite3

# Создадим sqlite3 базу с таблицей для сохранённого файла

sqlite_connection = sqlite3.connect('storage_of_bin_file.db')
dbh = sqlite_connection.cursor()

create_table_query = '''CREATE TABLE bin_file 
                        (
                        id INTEGER PRIMARY KEY,
                        name TEXT NOT NULL,
                        data BLOB
                        );'''

try:
    dbh.execute(create_table_query)
except:
    print("table bin_file already exists")

table bin_file already exists


In [288]:
sqlite_connection.commit()
dbh.close()

In [289]:
id = 0
file_name = 'nothing'

In [290]:
def store_bin_file_in_db(name, data):
    global id
    global sqlite_connection
    
    dbh = sqlite_connection.cursor()
    
    update_table_query = '''INSERT INTO bin_file
                          (id, name, data)  VALUES  (?, ?, ?)'''
    
    dbh.execute(update_table_query, (id, name, data))
    id += 1
    
    sqlite_connection.commit()
    
    print(f"{dbh.rowcount} rows updated")
    
    dbh.close()

Эмулируем ситуацию потерянных данных

In [310]:
FLAG_STORED_IN_DB = False

In [311]:
data = None # полученные данные потерялись

In [312]:
while (not FLAG_STORED_IN_DB):
    try:
        if data:
            # записываем данные в базу данных
            print("writing in db ...")
            
            store_bin_file_in_db(file_name, data)
            
            # записали успешно
            print("writing in db success")
            FLAG_STORED_IN_DB = True
        else:
            raise ValueError
    
    except(ValueError):
        fh = open('quasy_bin_file', 'rb')
        file_name = fh.name
        data = fh.read()
        fh.close()
    

writing in db ...
1 rows updated
writing in db success


In [294]:
def select_all_from_table():
    dbh = sqlite_connection.cursor()

    select_table_query = '''SELECT * FROM bin_file'''
    dbh.execute(select_table_query)

    rows_in_table = dbh.fetchall()
    dbh.close()
    
    return rows_in_table

In [295]:
def delete_all_from_table():
    global sqlite_connection
    dbh = sqlite_connection.cursor()
    
    delete_table_query = '''DELETE FROM  bin_file'''
    dbh.execute(delete_table_query)
    
    sqlite_connection.commit()
    
    print(f"{dbh.rowcount} rows updated")
    dbh.close()

In [296]:
#delete_all_from_table()

In [313]:
rows = select_all_from_table()

In [308]:
import pandas as pd

In [314]:
df = pd.DataFrame(rows, columns=['id', 'name', 'data'])
df

,id,name,data
0,0,quasy_bin_file,b'0b1111000b1000010b11001000b11011110b11000110...
1,1,quasy_bin_file,b'0b1111000b1000010b11001000b11011110b11000110...
2,2,quasy_bin_file,b'0b1111000b1000010b11001000b11011110b11000110...


#### 5.       Есть хранилище данных, которое обновляется раз в сутки, SQL или ElasticSearch (можно не знать про ES). Пользователь работает с хранилищем. Запрос выполнялся 30+ секунд. Результат корректный. Но этот же запрос может быть повторен другим пользователем. Как ускорить получение ответов от хранилища?

In [315]:
user_query = '''SELECT id, name FROM bin_file WHERE id > 1''' # условно долгий запрос

In [318]:
import time
from functools import lru_cache

In [326]:
@lru_cache
def execute_some_query(query):
    global sqlite_connection
    
    dbh = sqlite_connection.cursor()
    
    time.sleep(1)
    dbh.execute(user_query)
    
    rows = dbh.fetchall()
    dbh.close()
    
    return rows

In [327]:
%%time
execute_some_query(user_query) # первый запрос

Wall time: 1 s


[(2, 'quasy_bin_file')]

In [328]:
%%time
execute_some_query(user_query) # повторный запрос

Wall time: 0 ns


[(2, 'quasy_bin_file')]

Кэшируем запрос

# SQL

Вопрос по SQL
<br>
<br> 
<br>
В чем может быть отличие использование LIKE от MATCH … AGAINST?
<br>
Например в запросах
<br>
<br> 
<br>
<code>SELECT * FROM cache WHERE content like ('%future%' and '%trends%')</code>
<br>
<br> 
<br>
И
<br>
<br> 
<br>
<code>SELECT *  FROM cache WHERE  MATCH (content) AGAINST ( '+future +trends' IN BOOLEAN MODE)</code>

### Ответ:

LIKE будет дороже ведь происходит по шаблону, в отличии от MATCH AGAINST, который использует FULLTEXT индексы для сокращения времени затрачиваемого на поиск(но при этом индексы не обязательны).

<code>SELECT * FROM cache WHERE MATCH (content) AGAINST ( '+future +trends' IN BOOLEAN MODE)</code> :
<br>    
Все записи в которых содержаться слова ("future", "trends")<br>
futuretrends будет проигнорирован

In [364]:
test_query = '''SELECT * FROM cache WHERE content like '%future%' and content like '%trends%' '''

In [370]:
test_create_table_query = '''CREATE TABLE cache 
                            (
                            id INTEGER PRIMARY KEY,
                            content TEXT NOT NULL
                            );'''

dbh = sqlite_connection.cursor()
    
try:
    dbh.execute(test_create_table_query)
    sqlite_connection.commit()
except:
    print("table cache already exists")

test_update_query = '''
                    INSERT INTO cache
                    (id, content)  VALUES  (?, ?)
                    '''

dbh.execute('''DELETE FROM cache''')
sqlite_connection.commit()

dbh.execute(test_update_query, (0, "futuretrends"))
dbh.execute(test_update_query, (1, "future trends"))
dbh.execute(test_update_query, (2, "aaafuture trendsbbbb"))
dbh.execute(test_update_query, (3, "aaafuturewwwtrendsbbbb"))
dbh.execute(test_update_query, (4, "  future  trends "))
dbh.execute(test_update_query, (5, "nomatching"))

sqlite_connection.commit()



table cache already exists


In [371]:
dbh.execute('''SELECT * FROM cache''')
test_df = pd.DataFrame(dbh.fetchall(), columns = ['id', 'content'])

In [372]:
test_df

,id,content
0,0,futuretrends
1,1,future trends
2,2,aaafuture trendsbbbb
3,3,aaafuturewwwtrendsbbbb
4,4,future trends
5,5,nomatching


In [373]:
dbh.execute(test_query)
like_df = pd.DataFrame(dbh.fetchall(), columns = ['id', 'content'])

In [374]:
like_df

,id,content
0,0,futuretrends
1,1,future trends
2,2,aaafuture trendsbbbb
3,3,aaafuturewwwtrendsbbbb
4,4,future trends


In [375]:
dbh.close()

<code>SELECT * FROM cache WHERE content like '%future%' and content like '%trends%'</code><br>
Поменял запрос на такой потому что со скобками ничего не распознавало, возможно определённые СУБД понимают конструкцию " LIKE () ", но SQLite к сожалению нет.

Но в общем видно, что мы просто находим вхождения по шаблону, в то время как с полнотекстным поиском MATCH AGAINST мы бы получили записи с id=1 и id=4